# Extracting zipped parsed page xml files

This nb originally extracted the zip files containing the results of Isaac's parsing that were uploaded to the project [github](https://github.com/Southampton-Digital-Humanities/2023_Catalogue-Heading-Detection). Once we took over maintaining the code we've switched to processing the data locally so the zipping is obsolete. The majority of the code is now about parsing the xmls exported from Transkribus using updated versions of the code that Isaac wrote.

In [ ]:
import sys
if '../' not in sys.path:
    sys.path.append('../')
import shutil
import glob
import os
from xml.etree import ElementTree as ET
from tqdm import tqdm
from src.data import xml_extraction as xmle

### Extract zips

For when we were extracting zips of Isaac's data

In [ ]:
ic_data_path = "\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula\\split_data"

zips = glob.glob("\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula\\split_data\\*.zip")

# [shutil.unpack_archive(x, x[:-4], format="zip") for x in zips]

### Rename download jobs

In [ ]:
with open("..\\export_job_rename_dict.txt") as f:  # .txt was a result of massaging the pairings out of Rossitza's .xlsx
    rename_lines = f.readlines()
    rename_split = [x.split(":") for x in rename_lines]
    rename_dict = {x[0]:x[1].strip("\n") for x in rename_split}
    
rename_dict

In [ ]:
# for folder in done_zips:
#     renamed = os.path.join(os.path.dirname(folder), export_dict[os.path.basename(folder)])
#     os.rename(folder, renamed)

#### All lines in entry honoured as separate lines in output

This is a bit outdated now - I think Rossitza prefers to have all entries on one line for using in AntConc.

In [ ]:
target_zip = done_zips[10:11]
target_zip

In [ ]:
for f in target_zip:
    
    txts = glob.glob(os.path.join(f, "rawtextfiles\\*.txt"))
    raw_fname = os.path.join(f, "combinedrawtext.txt")
    
    with open(raw_fname, "w") as raw_out:
        for t in tqdm(txts):
            with open(t) as infile:
                entry = infile.read()                
                raw_out.write(entry)
                raw_out.write("\n\n")

In [ ]:
for f in target_zip:
    print(f"Combining  {f}")
    txts = glob.glob(os.path.join(f, "splittextfiles\\*.txt"))
    split_fname = os.path.join(f, "combinedsplittext.txt")
    
    with open(split_fname, "w") as split_out:
        for t in tqdm(txts):
            with open(t) as infile:
                entry = infile.readlines()
                dash_line = "-----------------------------------\n"
                non_english_line = "NON-ENGLISH SECTION LASTING"
                english_only_entry = [x for x in entry if dash_line not in x and non_english_line not in x]
                split_str = "".join(english_only_entry)
                split_out.write(split_str)
                split_out.write("\n\n")

#### All lines in entry concatenated onto one single line in output

Better - but we're now combining all the entries from one volume together as below

In [ ]:
for f in target_zip:
    
    txts = glob.glob(os.path.join(f, "rawtextfiles\\*.txt"))
    raw_fname = os.path.join(f, "combinedrawtext_single_line.txt")

    with open(raw_fname, "w") as raw_out:
        for t in tqdm(txts):
            with open(t) as infile:
                entry = infile.readlines()
                raw_str = " ".join([x.strip("\n") for x in entry])
                raw_out.write(raw_str + "\n")

In [ ]:
for f in target_zip:
    print(f"Combining  {f}")
    txts = glob.glob(os.path.join(f, "splittextfiles\\*.txt"))
    split_fname = os.path.join(f, "combinedsplittext_single_line.txt")
    
    with open(split_fname, "w") as split_out:
        for t in tqdm(txts):
            with open(t) as infile:
                entry = infile.readlines()
                dash_line = "-----------------------------------\n"
                non_english_line = "NON-ENGLISH SECTION LASTING"
                english_only_entry = [x.strip("\n") for x in entry if dash_line not in x and non_english_line not in x]
                split_str = " ".join(english_only_entry)
                split_out.write(split_str + "\n")

### Combined volumes

In [ ]:
# paired globs
combined_volumes = []
for i in range(1):
    network_loc = "\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula"
    suffix = "column pages Transkribus export"
    combined_volumes.append([os.path.join(network_loc, f"BMC_{i+1} 2 {suffix}"), os.path.join(network_loc, f"BMC_{i+1} 4 {suffix}")])    
    
#     When running locally
#     combined_volumes.append([f'C:\\Users\\HLloyd\\Documents\\BMC_{i+1}_2', f'C:\\Users\\HLloyd\\Documents\\BMC_{i+1}_4'])

In [ ]:
combined_volumes

#### Raw text

In [ ]:
for i, v in enumerate(combined_volumes):
    print(f"Combining  {os.path.basename(v[0])}, {os.path.basename(v[1])}")
    txts = glob.glob(os.path.join(v[0], "rawtextfiles\\*.txt")) + glob.glob(os.path.join(v[1], "rawtextfiles\\*.txt"))
    single_vol_dir = os.path.join(os.path.dirname(v[0]), "split_data\\test", f"BMC_{i+1}")
    if not os.path.exists(single_vol_dir):
        os.mkdir(single_vol_dir)
    raw_fname = os.path.join(single_vol_dir, "combinedrawtext_single_line.txt")
    
#     with open(raw_fname, "w", encoding="utf-8-sig") as raw_out:
#         for t in tqdm(txts):
#             with open(t, encoding="utf-8-sig") as infile:
#                 entry = infile.readlines()
#                 raw_str = " ".join([x.strip("\n") for x in entry])
#                 raw_out.write(raw_str + "\n")

In [ ]:
txts

#### Split text

In [ ]:
for i, v in enumerate(combined_volumes):
    print(f"Combining  {v[0]}")
    txts = glob.glob(os.path.join(v[0], "splittextfiles\\*.txt")) + glob.glob(os.path.join(v[1], "splittextfiles\\*.txt"))
    single_vol_dir = os.path.join(os.path.dirname(v[0]), f"BMC_{i+1}")
    if not os.path.exists(single_vol_dir):
        os.mkdir(single_vol_dir)
    split_fname = os.path.join(single_vol_dir, "combinedsplittext_single_line.txt")
    
    with open(split_fname, "w", encoding="utf-8-sig") as split_out:
        for t in tqdm(txts):
            with open(t, encoding="utf-8-sig") as infile:
                entry = infile.readlines()
                dash_line = "-----------------------------------\n"
                non_english_line = "NON-ENGLISH SECTION LASTING"
                english_only_entry = [x.strip("\n") for x in entry if dash_line not in x and non_english_line not in x]
                split_str = " ".join(english_only_entry)
                split_out.write(split_str + "\n")

### Refactor output check

In [ ]:
refactor_out = glob.glob(r"C:\Users\HLloyd\Documents\BMC_*")

In [ ]:
refactor_out

In [ ]:
for x in refactor_out:
    shutil.copytree(x, os.path.join(ic_data_path, os.path.basename(x)))

In [ ]:
refactor_sizes = {x: [os.path.getsize(y) for y in sorted(glob.glob(os.path.join(x, "*.txt")))] for x in refactor_out}

In [ ]:
id_out = glob.glob(r"C:\Users\HLloyd\Documents\Incunabula-Catalogue-Entry-Detection\data\*\rawtextfiles")

In [ ]:
id_out

In [ ]:
id_paired_sizes = {x: [os.path.getsize(y) for y in sorted(glob.glob(os.path.join(x, "*.txt"))) if os.path.exists(os.path.join('C:\\Users\\HLloyd\\Documents\\', y[68:]))] for x in id_out}

In [ ]:
# refactor_sizes.pop(r"C:\Users\HLloyd\Documents\BMC_5_2\rawtextfiles")
for (me_k, me_v), (u_k, u_v) in zip(refactor_sizes.items(), id_paired_sizes.items()):
    check = [x == y for x, y in zip(me_v, u_v)]
    print(f"{me_k[-25:-10]}: {sum(check) == len(check)}")

In [ ]:
zips = glob.glob(r"C:\Users\HLloyd\Documents\BMC*")

In [ ]:
zips

In [ ]:
[shutil.unpack_archive(x, os.path.join(os.path.dirname(x), os.path.basename(x[:-4]))) for x in zips[1:]]

## Running main.py to debug

In [ ]:
page_xmls = []
for i in range(8, 9):
    network_loc = "\\\\ad\\collections\\TwoCenturies\\TwoCenturies IV\\Incunabula"
    suffix = "column pages Transkribus export"
    page_xmls.append(
        [
            os.path.join(network_loc, f"BMC_{i + 1} 2 {suffix}"),
            os.path.join(network_loc, f"BMC_{i + 1} 4 {suffix}")
        ]
    )

# this one didn't work for some reason
# 'BMC_5 2 column pages Transkribus export'

Use 8 as the test bed as it's got one of the lowest numbers of pages

In [ ]:
for x in page_xmls:
    pageXMLLocation_2 = os.path.join(x[0], r"*\*\page\*.xml")
    pageXMLLocation_4 = os.path.join(x[1], r"*\*\page\*.xml")
    out_path = os.path.join(network_loc, "split_data\\test", os.path.basename(x[0]).split(" ")[0])
    attempts = 0
    while attempts < 3:
        xmls_2 = glob.glob(os.fsencode(pageXMLLocation_2))
        xmls_4 = glob.glob(os.fsencode(pageXMLLocation_4))

        if xmls_2 and xmls_4:
            xmls = xmls_2 + xmls_4
            break
        else:
            attempts += 1
            continue
    else:
        raise IOError(f"Failed to connect to {os.path.dirname(pageXMLLocation_2)}  {os.path.basename(pageXMLLocation_2)}/{os.path.basename(pageXMLLocation_4)}")
    
    xmlroots = {}

    print(f"\nGetting xml roots from {os.path.dirname(pageXMLLocation_2)}  {os.path.basename(pageXMLLocation_2)}/{os.path.basename(pageXMLLocation_4)}")
    for file in tqdm(xmls):
        fileName = os.fsdecode(file)
        attempts = 0
        while attempts < 3:
            try:
                tree = ET.parse(fileName)
                break
            except FileNotFoundError:
                attempts += 1
                continue
        else:
            raise FileNotFoundError(f"Failed to connect to: {fileName}")
        root = tree.getroot()
        xmlroots[fileName.split("_")[-1][:-4]] = root

In [ ]:
xmlroots['0042']

In [ ]:
print("\nExtrating catalogue entries from xmls")
currentVolume = {k:v for k,v in sorted(xmlroots.items())}
allLines = xmle.extractLinesForVol(list(currentVolume.values()))
allLines = [line for line in allLines if line is not None]
titles, allTitleIndices = xmle.findHeadings(allLines)
titleRefs = xmle.genTitleRefs(allTitleIndices, allLines)

In [ ]:
def saveAll(currentVolume, directory, allTitleIndices, allLines, path, titleRefs):
    # out_path = os.path.join(path, "generated")  # if you do want them to go into a sub_folder of the output loc
    out_path = path
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    xmle.savePoorlyScannedPages(xmle.getPoorlyScannedPages(currentVolume, directory), out_path)
    print("Saving raw txt files")
    xmle.saveRawTxt(allTitleIndices, allLines, os.path.join(out_path, "rawtextfiles"), titleRefs)
#     print("Saving split txt files")
#     xmle.saveSplitTxt(allTitleIndices, allLines, os.path.join(out_path, "splittextfiles"), titleRefs)
    xmle.saveXML(allTitleIndices, allLines, out_path, titleRefs)

In [ ]:
print(f"\nSaving catalogue entries to {out_path}\n")
saveAll(
    currentVolume=list(currentVolume.values()),
    directory=xmls,
    allTitleIndices=allTitleIndices,
    allLines=allLines,
    path=out_path,
    titleRefs=titleRefs
)